# Демонстация использования модели

### Подготовим среду выполнения

In [7]:
import torch

if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P4


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
!pip install transformers

### Загрузим токенайзер

In [10]:
PATH_TO_PRETRAINED_TOKENIZER = 'sberbank-ai/sbert_large_nlu_ru'

from transformers import AutoTokenizer, AutoModel

sbertTokenizer = AutoTokenizer.from_pretrained(PATH_TO_PRETRAINED_TOKENIZER)

### Объявим класс дистиллированной модели и загрузим модель

In [11]:
from torch import nn
from torch.autograd import Variable

class SimpleLSTM(nn.Module):

    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, n_layers,
                 bidirectional, dropout, batch_size, device=None):
        super(SimpleLSTM, self).__init__()
        self.batch_size = batch_size
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(input_dim, embedding_dim)

        self.rnn = nn.LSTM(embedding_dim,
                           hidden_dim,
                           num_layers=n_layers,
                           bidirectional=bidirectional,
                           dropout=dropout)

        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(dropout)
        self.device = self.init_device(device)
        self.hidden = self.init_hidden()

    @staticmethod
    def init_device(device):
        if device is None:
            return torch.device('cuda')
        return device

    def init_hidden(self):
        return (Variable(torch.zeros(2 * self.n_layers, self.batch_size, self.hidden_dim).to(self.device)),
                Variable(torch.zeros(2 * self.n_layers, self.batch_size, self.hidden_dim).to(self.device)))

    def forward(self, text, text_lengths=None):
        self.hidden = self.init_hidden()
        x = self.embedding(text)
        x, self.hidden = self.rnn(x, self.hidden)
        hidden, cell = self.hidden
        hidden = self.dropout(torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1))
        x = self.fc(hidden)
        return x

In [12]:
modelParameters = {'input_dim' : sbertTokenizer.vocab_size,
                   'embedding_dim' : 50,
                   'hidden_dim' : 256,
                   'output_dim' : 1024,
                   'n_layers' : 2,
                   'bidirectional' : True,
                   'dropout' : 0.2,
                   'batch_size' : 32}

model = SimpleLSTM(
            input_dim = modelParameters['input_dim'],
            embedding_dim = modelParameters['embedding_dim'],
            hidden_dim = modelParameters['hidden_dim'],
            output_dim = modelParameters['output_dim'],
            n_layers = modelParameters['n_layers'],
            bidirectional = modelParameters['bidirectional'],
            dropout = modelParameters['dropout'],
            batch_size = modelParameters['batch_size'])

In [13]:
import torch.nn as nn
import torch.nn.functional as F

class DistilledModel(nn.Module):
    def __init__(self):
        super(DistilledModel, self).__init__()
        self.model = model
        self.dropout = nn.Dropout(p=0.1)
        self.lin = nn.Linear(1024, 10)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x, text_lengths=None):
        x = self.model(x.t(), text_lengths=text_lengths).squeeze(1)
        x = self.dropout(x)
        x = self.lin(x)
        x = self.softmax(x)
        return x

In [14]:
PATH_TO_MODEL = '/content/drive/My Drive/distilled-model'

distilledModel = DistilledModel()

distilledModel.load_state_dict(torch.load(PATH_TO_MODEL))

distilledModel.to(device)

DistilledModel(
  (model): SimpleLSTM(
    (embedding): Embedding(120138, 50)
    (rnn): LSTM(50, 256, num_layers=2, dropout=0.2, bidirectional=True)
    (fc): Linear(in_features=512, out_features=1024, bias=True)
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (dropout): Dropout(p=0.1, inplace=False)
  (lin): Linear(in_features=1024, out_features=10, bias=True)
  (softmax): Softmax(dim=1)
)

### Предскажем темы для некоторых предложений

In [15]:
# Этот массив должен быть именно таким. В случае, если вы хотите предсказывать другие темы, необходимо заново дообучить модель в другой тетрадке.
id_to_topic = ['Из жизни','Россия','Наука и техника','Культура','Бывший СССР','Экономика','Бизнес','Спорт','Путешествия','Интернет и СМИ']

In [16]:
def predict(model, tokenizer, sent, id_to_topic):
        tokenized_sent = tokenizer([sent] * 32, padding=True, truncation=True, max_length=20)
        inds = torch.tensor(tokenized_sent['input_ids'])
        inds_cuda = inds.to(device)
        model.eval()
        model_output = model(inds_cuda, None)[0]
        _, predicted = torch.max(model_output, 0)
        return id_to_topic[predicted]

In [17]:
sents = [
         'Инфляция - жуткая штука',
         'В Большом театре без перемен',
         'Суперкомпьютеры определяют будущее',
         'Футболисты выиграли матч'
]

In [18]:
for sent in sents:
    predicted = predict(distilledModel, sbertTokenizer, sent, id_to_topic)
    print('Предложение \'%s\'  имеет тему \'%s\'' % (sent, predicted))

Предложение 'Инфляция - жуткая штука'  имеет тему 'Экономика'
Предложение 'В Большом театре без перемен'  имеет тему 'Культура'
Предложение 'Суперкомпьютеры определяют будущее'  имеет тему 'Наука и техника'
Предложение 'Футболисты выиграли матч'  имеет тему 'Спорт'


In [21]:
tokenized_sent = sbertTokenizer('Футболисты выиграли матч', padding=True, truncation=True, max_length=20); tokenized_sent

{'input_ids': [101, 20236, 19672, 5071, 102], 'token_type_ids': [0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1]}